In [53]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd
import numpy as np

In [54]:
path =('../../dataset/')

In [3]:
movies_data = pd.read_csv(path + 'movies_crawling_sample_withNaN.csv')
movies_data.head()

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A little boy named Andy loves to be in his roo...,United States,English,John Lasseter,1995.0,1 hour 21 minutes,All,toy,toy comes to life,rivalry
1,2,Jumanji (1995),Adventure|Children|Fantasy,"Jumanji, one of the most unique--and dangerous...",United States,"English, French",Joe Johnston,1995.0,1 hour 44 minutes,All,board game,trapped in a game,adventurer
2,3,Grumpier Old Men (1995),Comedy|Romance,Things don't seem to change much in Wabasha Co...,United States,"English, Italian, German",Howard Deutch,1995.0,1 hour 41 minutes,15,sequel,boat,lake
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,This story based on the best selling novel by ...,United States,English,Forest Whitaker,1995.0,2 hours 4 minutes,18,black american,friendship between women,husband wife relationship
4,5,Father of the Bride Part II (1995),Comedy,"In this sequel to ""Father of the Bride"", Georg...",United States,English,Charles Shyer,1995.0,1 hour 46 minutes,All,fatherhood,doberman,dog


In [58]:
movies_data.shape

(9742, 13)

In [125]:
links_data = pd.read_csv(path+'links.csv')
movies = pd.merge(movies_data, links_data, on='movieId')

크롤링 링크 만들기

In [57]:
def create_imdb_url(imdb_id):
    # imdbId를 문자열로 변환하고, 7자리가 되도록 앞에 0을 추가
    imdb_id_str = str(imdb_id).zfill(7)
    # URL 생성
    url = f"https://www.imdb.com/title/tt{imdb_id_str}/"
    return url

## 현재 결측치 확인하기

In [7]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       9742 non-null   int64  
 1   title         9742 non-null   object 
 2   genres        9742 non-null   object 
 3   storyline     9729 non-null   object 
 4   country       9735 non-null   object 
 5   language      9719 non-null   object 
 6   director      9735 non-null   object 
 7   release_year  9359 non-null   float64
 8   runtime       9740 non-null   object 
 9   age_rating    9504 non-null   object 
 10  tag_1         9441 non-null   object 
 11  tag_2         9406 non-null   object 
 12  tag_3         9388 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 989.5+ KB


결측치 적은 거부터 처리해보자

1. runtime
2. director
3. country
4. language

### 1. runtime

In [11]:
# 'column_name' 열에서 null 값인 행만 선택
null_rows = movies_data[movies_data['runtime'].isnull()]
null_rows

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
500,580,Princess Caraboo (1994),Drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1500,2025,Lolita (1997),Drama|Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
browser = webdriver.Chrome()
browser.get('https://www.google.com/')

soup = BeautifulSoup(browser.page_source, 'html.parser')

names = null_rows['title'].tolist()

In [15]:
from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

#un_storyline = []
#un_country = []
#un_language = []
#un_director = []
#un_release_year = []
#un_runtime = []
#un_age_rating = []
#un_tags=[]
for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(1)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(2)
    
    ################################################################################

    soup = BeautifulSoup(browser.page_source, 'html.parser')

    ################################################################################

    # storyline
    try:
        storyline_text = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
        storyline = storyline_text[0].text

        movies_data.loc[movies_data['title'] == name, 'storyline'] = storyline

    except IndexError:
        un_storyline.append(name)
        
    ################################################################################
    #age_rating
    try:
        age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')
        movies_data.loc[movies_data['title'] == name, 'age_rating'] = age_rating[0].text

    except:
        un_age_rating.append(name)
        
     ################################################################################
    # tags
    try:
        tags = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-chip-list--base.ipc-chip-list.sc-e5ec7f64-0.iPSjrC > div.ipc-chip-list__scroller > a > span')
        movies_data.loc[movies_data['title'] == name, 'tag_1'] = tags[0].text
        movies_data.loc[movies_data['title'] == name, 'tag_2'] = tags[1].text
        movies_data.loc[movies_data['title'] == name, 'tag_3'] = tags[2].text
        
    except:
        un_tags.append(name)

    ################################################################################
            
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(1.5)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    

    #country
    try:
        country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
        movies_data.loc[movies_data['title'] == name, 'country'] = country.text

    except:
        un_country.append(name)
        
   ################################################################################
    # language
    try:
        language_select = ''
        language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')
    
        # 모든 <ul> 태그를 순회
        for ul in language_select:
            # 각 <ul> 태그 내의 모든 <li> 태그를 순회
            for li in ul.find_all('li'):
                span = li.find('span', class_='ipc-metadata-list-item__label')
                if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
                    # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
                    languages = [a.get_text(strip=True) for a in li.find_all('a')]
                    languages_str = ', '.join(languages)
                    break
            else:
                # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
                continue
            # 'Language' 라벨을 찾은 경우, 반복문 종료
            break
        
        movies_data.loc[movies_data['title'] == name, 'language'] = languages_str

    except:
        un_language.append(name)
        
     ################################################################################
    # director
    try:
        director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'director'] = director

    except:
        un_director.append(name)
        
     ################################################################################
    #release_year
    try:
        release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'release_year'] = release_year

    except:
        un_release_year.append(name)
        
     ################################################################################
    #runtime
    try:

        temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
        for tag in temp:
            if 'hour' in tag.text or 'minutes' in tag.text:
                runtime = tag.text
        
        movies_data.loc[movies_data['title'] == name, 'runtime'] = runtime

    except:
        un_runtime.append(name)
        

    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies: 100%|████████████████████████████████████████████████████████████████| 2/2 [00:34<00:00, 17.41s/movie]


In [17]:
movies_data[movies_data['runtime'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3


### 2. director

In [18]:
null_rows = movies_data[movies_data['director'].isnull()]
null_rows

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
585,720,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy,NaN,NaN,English,NaN,NaN,1 hour 57 minutes,NaN,NaN,NaN,NaN
872,1161,"Tin Drum, The (Blechtrommel, Die) (1979)",Drama|War,NaN,NaN,Italian,NaN,NaN,1 hour 24 minutes,NaN,NaN,NaN,NaN
7600,86668,Louis Theroux: Law & Disorder (2008),Documentary,NaN,NaN,"English, Portuguese, Spanish, Italian, French",NaN,NaN,2 hours 10 minutes,NaN,NaN,NaN,NaN
7924,95497,Dragon Ball Z: Super Android 13! (Doragon bôru...,Action|Adventure|Animation,The death of Dr. Gero at the hands of Androids...,NaN,"Japanese, English",NaN,NaN,47 minutes,TV-PG,scream,dragon ball z,goku character
8589,117511,Hello Ladies: The Movie (2014),Comedy,NaN,NaN,English,NaN,NaN,2 hours 2 minutes,NaN,NaN,NaN,NaN


In [34]:
names = null_rows['title'].tolist()

from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

un_storyline = []
un_country = []
un_language = []
un_director = []
un_release_year = []
un_runtime = []
un_age_rating = []
un_tags=[]

for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(1)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(2)
    
    ################################################################################

    soup = BeautifulSoup(browser.page_source, 'html.parser')

    ################################################################################

    # storyline
    try:
        storyline_text = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
        storyline = storyline_text[0].text

        movies_data.loc[movies_data['title'] == name, 'storyline'] = storyline

    except IndexError:
        un_storyline.append(name)
        
    ################################################################################
    #age_rating
    try:
        age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')
        movies_data.loc[movies_data['title'] == name, 'age_rating'] = age_rating[0].text

    except:
        un_age_rating.append(name)
        
     ################################################################################
    # tags
    try:
        tags = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-chip-list--base.ipc-chip-list.sc-e5ec7f64-0.iPSjrC > div.ipc-chip-list__scroller > a > span')
        movies_data.loc[movies_data['title'] == name, 'tag_1'] = tags[0].text
        movies_data.loc[movies_data['title'] == name, 'tag_2'] = tags[1].text
        movies_data.loc[movies_data['title'] == name, 'tag_3'] = tags[2].text
        
    except:
        un_tags.append(name)

    ################################################################################
            
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(1.5)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    

    #country
    try:
        country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
        movies_data.loc[movies_data['title'] == name, 'country'] = country.text

    except:
        un_country.append(name)
        
   ################################################################################
    # language
    try:
        language_select = ''
        language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')
    
        # 모든 <ul> 태그를 순회
        for ul in language_select:
            # 각 <ul> 태그 내의 모든 <li> 태그를 순회
            for li in ul.find_all('li'):
                span = li.find('span', class_='ipc-metadata-list-item__label')
                if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
                    # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
                    languages = [a.get_text(strip=True) for a in li.find_all('a')]
                    languages_str = ', '.join(languages)
                    break
            else:
                # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
                continue
            # 'Language' 라벨을 찾은 경우, 반복문 종료
            break
        
        movies_data.loc[movies_data['title'] == name, 'language'] = languages_str

    except:
        un_language.append(name)
        
     ################################################################################
    # director
    try:
        director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'director'] = director

    except:
        un_director.append(name)
        
     ################################################################################
    #release_year
    try:
        release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'release_year'] = release_year

    except:
        un_release_year.append(name)
        
     ################################################################################
    #runtime
    try:

        temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
        for tag in temp:
            if 'hour' in tag.text or 'minutes' in tag.text:
                runtime = tag.text
        
        movies_data.loc[movies_data['title'] == name, 'runtime'] = runtime

    except:
        un_runtime.append(name)
        

    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies: 100%|██████████████████████████████████████████████████████████████| 21/21 [05:47<00:00, 16.54s/movie]


In [24]:
names[0]

'Wallace & Gromit: The Best of Aardman Animation (1996)'

In [31]:
movies_data[movies_data['director'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3


In [29]:
storyline = 'One of the installments in the Louis Theroux Law and Disorder Series'
age_rating = 'Not Rated'
country = 'United Kingdom'
language = 'English'
director = 'Emma Cooper'
release_year = '2008'
runtime = '2 hours'

In [30]:
movies_data.loc[movies_data['movieId'] == 86668, 'storyline'] = storyline
movies_data.loc[movies_data['movieId'] == 86668, 'age_rating'] = age_rating
movies_data.loc[movies_data['movieId'] == 86668, 'country'] = country
movies_data.loc[movies_data['movieId'] == 86668, 'language'] = language
movies_data.loc[movies_data['movieId'] == 86668, 'director'] = director
movies_data.loc[movies_data['movieId'] == 86668, 'release_year'] = release_year
movies_data.loc[movies_data['movieId'] == 86668, 'runtime'] = runtime

### 3. country

In [33]:
null_rows = movies_data[movies_data['language'].isnull()]
null_rows

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
1660,2227,"Lodger: A Story of the London Fog, The (1927)",Crime|Drama|Thriller,"A serial killer known as ""The Avenger"" is on t...",United Kingdom,NaN,Alfred Hitchcock,1927.0,1 hour 32 minutes,15,serial killer,lodger,detective
2279,3022,"General, The (1926)",Comedy|War,"Johnnie loves his train (""The General"") and An...",United States,NaN,Clyde Bruckman,1926.0,1 hour 18 minutes,All,train,confederate army,civil war
2739,3677,Baraka (1992),Documentary,"Without words, cameras show us the world, with...",United States,NaN,Ron Fricke,1992.0,1 hour 36 minutes,12,human nature,environmental education,world culture
3294,4454,More (1998),Animation|Drama|Sci-Fi|IMAX,"A factory worker in a dark, gray world assembl...",United States,NaN,Mark Osborne,1998.0,6 minutes,Not Rated,claymation,stop motion animation,one word title
4386,6433,"Man with the Movie Camera, The (Chelovek s kin...",Documentary,This playful film is at once a documentary of ...,Soviet Union,NaN,Dziga Vertov,1929.0,1 hour 8 minutes,Not Rated,experimental film,avant garde,soviet union
6097,42191,Luxo Jr. (1986),Animation|Children,"A larger lamp watches while a smaller, younger...",United States,NaN,John Lasseter,1986.0,2 minutes,All,no dialogue,lamp,ball
6431,51573,Meshes of the Afternoon (1943),Fantasy,"On her way home, a solitary woman picks up a f...",United States,NaN,Maya Deren,1943.0,14 minutes,Not Rated,national film registry,flower,knife
6562,55061,Electroma (2006),Drama|Sci-Fi,Daft Punk's Electroma is an odyssey of two rob...,France,NaN,Thomas Bangalter,2006.0,1 hour 14 minutes,TV-PG,robot sci fi,existentialism,cyberpunk
7172,72104,Balance (1989),Animation|Drama|Mystery|Sci-Fi|Thriller,The setting is on a floating platform where a ...,Germany,NaN,Christoph Lauenstein,1989.0,7 minutes,NaN,balance,surrealism,stop motion
7184,72356,Partly Cloudy (2009),Animation|Children|Comedy|Fantasy,"Everyone knows that the stork delivers babies,...",United States,NaN,Peter Sohn,2009.0,6 minutes,G,cg animation,delivery stork,cgi animation


크롤링은 2 코드를 이용해서 진행

In [35]:
movies_data[movies_data['language'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3


<hr>

# 다시 결측치 확인

In [36]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       9742 non-null   int64 
 1   title         9742 non-null   object
 2   genres        9742 non-null   object
 3   storyline     9735 non-null   object
 4   country       9742 non-null   object
 5   language      9742 non-null   object
 6   director      9742 non-null   object
 7   release_year  9366 non-null   object
 8   runtime       9742 non-null   object
 9   age_rating    9510 non-null   object
 10  tag_1         9444 non-null   object
 11  tag_2         9409 non-null   object
 12  tag_3         9391 non-null   object
dtypes: int64(1), object(12)
memory usage: 989.5+ KB


1. storyline → 9735
2. release_year → 9366
3. age_rating → 9510
4. tag → 9391

### 1. storyline

In [37]:
null_rows = movies_data[movies_data['storyline'].isnull()]
null_rows

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
624,791,"Last Klezmer: Leopold Kozlowski, His Life and ...",Documentary,NaN,United States,"English, Polish, Yiddish, Russian",Yale Strom,1994.0,1 hour 25 minutes,NaN,portrait,klezmer music,independent film
708,927,"Women, The (1939)",Comedy,NaN,United States,"English, French",George Cukor,1939.0,2 hours 13 minutes,NaN,NaN,NaN,NaN
938,1238,Local Hero (1983),Comedy,NaN,United Kingdom,"English, Russian, Japanese",Bill Forsyth,1983.0,1 hour 51 minutes,NaN,NaN,NaN,NaN
7231,73501,Pekka ja Pätkä Suezilla (1958),Comedy,NaN,Finland,Finnish,Armand Lohikoski,1958.0,1 hour 14 minutes,NaN,united nations mission,africa,rebel
8621,118784,Good Copy Bad Copy (2007),Documentary,NaN,Denmark,English,Ralf Christensen,2007.0,59 minutes,NaN,copy,NaN,NaN
9600,175781,Der Herr Karl (1961),Comedy,NaN,Austria,German,Erich Neuberg,NaN,55 minutes,NaN,storeroom,storekeeper,reminiscence
9655,180777,Die Frauen von Ravensbrück (2005),Documentary,NaN,Germany,German,Loretta Walz,NaN,1 hour 30 minutes,NaN,f rated,NaN,NaN


In [41]:
names = null_rows['title'].tolist()

from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

un_storyline = []
un_country = []
un_language = []
un_director = []
un_release_year = []
un_runtime = []
un_age_rating = []
un_tags=[]

for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(1)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(2)
    
    ################################################################################

    soup = BeautifulSoup(browser.page_source, 'html.parser')

    ################################################################################

    # storyline
    try:
        storyline_text = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
        storyline = storyline_text[0].text

        movies_data.loc[movies_data['title'] == name, 'storyline'] = storyline

    except IndexError:
        un_storyline.append(name)
        
    ################################################################################
    #age_rating
    try:
        age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')
        movies_data.loc[movies_data['title'] == name, 'age_rating'] = age_rating[0].text

    except:
        un_age_rating.append(name)
        
     ################################################################################
    # tags
    try:
        tags = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-chip-list--base.ipc-chip-list.sc-e5ec7f64-0.iPSjrC > div.ipc-chip-list__scroller > a > span')
        movies_data.loc[movies_data['title'] == name, 'tag_1'] = tags[0].text
        movies_data.loc[movies_data['title'] == name, 'tag_2'] = tags[1].text
        movies_data.loc[movies_data['title'] == name, 'tag_3'] = tags[2].text
        
    except:
        un_tags.append(name)

    ################################################################################
            
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(1.5)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    

    #country
    try:
        country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
        movies_data.loc[movies_data['title'] == name, 'country'] = country.text

    except:
        un_country.append(name)
        
   ################################################################################
    # language
    try:
        language_select = ''
        language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')
    
        # 모든 <ul> 태그를 순회
        for ul in language_select:
            # 각 <ul> 태그 내의 모든 <li> 태그를 순회
            for li in ul.find_all('li'):
                span = li.find('span', class_='ipc-metadata-list-item__label')
                if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
                    # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
                    languages = [a.get_text(strip=True) for a in li.find_all('a')]
                    languages_str = ', '.join(languages)
                    break
            else:
                # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
                continue
            # 'Language' 라벨을 찾은 경우, 반복문 종료
            break
        
        movies_data.loc[movies_data['title'] == name, 'language'] = languages_str

    except:
        un_language.append(name)
        
     ################################################################################
    # director
    try:
        director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'director'] = director

    except:
        un_director.append(name)
        
     ################################################################################
    #release_year
    try:
        release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'release_year'] = release_year

    except:
        un_release_year.append(name)
        
     ################################################################################
    #runtime
    try:

        temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
        for tag in temp:
            if 'hour' in tag.text or 'minutes' in tag.text:
                runtime = tag.text
        
        movies_data.loc[movies_data['title'] == name, 'runtime'] = runtime

    except:
        un_runtime.append(name)
        

    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies: 100%|████████████████████████████████████████████████████████████████| 6/6 [01:20<00:00, 13.40s/movie]


In [66]:
movies_data[movies_data['storyline'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3


In [49]:
storyline = 'Years after the Holocaust, survivor Leopold Kozlowski lives in Poland, and is one of the few remaining experts in the Klezmer tradition of music, one played predominantly by the Eastern European Jewish community. Director Yael Strom follows Kozlowski as he travels back to his village in the Ukraine, where he visits his father\'s grave and speaks about his relatives who perished in the Holocaust. Strom honors Kozlowski\'s life as a teacher, musician and survivor'
age_rating = 'Not Rated'
#country = 'United Kingdom'
#language = 'English'
#director = 'Emma Cooper'
#release_year = '2008'
#runtime = '2 hours'

In [50]:
movies_data.loc[movies_data['movieId'] == 791, 'storyline'] = storyline
movies_data.loc[movies_data['movieId'] == 791, 'age_rating'] = age_rating

In [52]:
movies_data.to_csv("temp_result.csv", index = False)

______________________________________________________________________________________________________

이어서 다시 시작!

In [56]:
movies_data = pd.read_csv('temp_result.csv')
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       9742 non-null   int64  
 1   title         9742 non-null   object 
 2   genres        9742 non-null   object 
 3   storyline     9740 non-null   object 
 4   country       9742 non-null   object 
 5   language      9721 non-null   object 
 6   director      9742 non-null   object 
 7   release_year  9366 non-null   float64
 8   runtime       9742 non-null   object 
 9   age_rating    9515 non-null   object 
 10  tag_1         9446 non-null   object 
 11  tag_2         9411 non-null   object 
 12  tag_3         9393 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 989.5+ KB


1. storyline
2. language
3. release_year
4. age_rating
5. tag

In [60]:
null_rows = movies_data[movies_data['storyline'].isnull()]

browser = webdriver.Chrome()
browser.get('https://www.google.com/')

soup = BeautifulSoup(browser.page_source, 'html.parser')

names = null_rows['title'].tolist()

In [64]:
storyline = 'Im Zweiten Weltkrieg waren mehr als 100.000 Frauen aus Ländern in West- und Osteuropa in Ravensbrück inhaftiert. Zehntausende starben.Bei denen, die die Haft überlebten, hinterließen Lager, Todesbedrohung, Hunger und Seuchen bleibende Schäden an Körper und Seele.48 Frauen erzählen in dem Film von der entwürdigenden Ankunft, dem Zählappell, der Zwangsarbeit, medizinischen Experimenten, den Sterilisationen und vom Hunger.'
age_rating = 'Not Rated'
#country = 'United Kingdom'
#language = 'English'
#director = 'Emma Cooper'
release_year = '2005'
#runtime = '2 hours'

In [65]:
movies_data.loc[movies_data['movieId'] == 180777, 'storyline'] = storyline
movies_data.loc[movies_data['movieId'] == 180777, 'age_rating'] = age_rating
movies_data.loc[movies_data['movieId'] == 180777, 'release_year'] = release_year

2. language

In [67]:
null_rows = movies_data[movies_data['language'].isnull()]
null_rows

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
1660,2227,"Lodger: A Story of the London Fog, The (1927)",Crime|Drama|Thriller,"A serial killer known as ""The Avenger"" is on t...",United Kingdom,NaN,Alfred Hitchcock,1927.0,1 hour 32 minutes,15,serial killer,lodger,detective
2279,3022,"General, The (1926)",Comedy|War,"Johnnie loves his train (""The General"") and An...",United States,NaN,Clyde Bruckman,1926.0,1 hour 18 minutes,All,train,confederate army,civil war
2739,3677,Baraka (1992),Documentary,"Without words, cameras show us the world, with...",United States,NaN,Ron Fricke,1992.0,1 hour 36 minutes,12,human nature,environmental education,world culture
3294,4454,More (1998),Animation|Drama|Sci-Fi|IMAX,"A factory worker in a dark, gray world assembl...",United States,NaN,Mark Osborne,1998.0,6 minutes,Not Rated,claymation,stop motion animation,one word title
4386,6433,"Man with the Movie Camera, The (Chelovek s kin...",Documentary,This playful film is at once a documentary of ...,Soviet Union,NaN,Dziga Vertov,1929.0,1 hour 8 minutes,Not Rated,experimental film,avant garde,soviet union
6097,42191,Luxo Jr. (1986),Animation|Children,"A larger lamp watches while a smaller, younger...",United States,NaN,John Lasseter,1986.0,2 minutes,All,no dialogue,lamp,ball
6431,51573,Meshes of the Afternoon (1943),Fantasy,"On her way home, a solitary woman picks up a f...",United States,NaN,Maya Deren,1943.0,14 minutes,Not Rated,national film registry,flower,knife
6562,55061,Electroma (2006),Drama|Sci-Fi,Daft Punk's Electroma is an odyssey of two rob...,France,NaN,Thomas Bangalter,2006.0,1 hour 14 minutes,TV-PG,robot sci fi,existentialism,cyberpunk
7172,72104,Balance (1989),Animation|Drama|Mystery|Sci-Fi|Thriller,The setting is on a floating platform where a ...,Germany,NaN,Christoph Lauenstein,1989.0,7 minutes,NaN,balance,surrealism,stop motion
7184,72356,Partly Cloudy (2009),Animation|Children|Comedy|Fantasy,"Everyone knows that the stork delivers babies,...",United States,NaN,Peter Sohn,2009.0,6 minutes,G,cg animation,delivery stork,cgi animation


In [113]:
names = null_rows['title'].tolist()

from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

un_storyline = []
un_country = []
un_language = []
un_director = []
un_release_year = []
un_runtime = []
un_age_rating = []
un_tags=[]

for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(1)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(2)
    
    ################################################################################

    soup = BeautifulSoup(browser.page_source, 'html.parser')

    ################################################################################

    # storyline
    try:
        storyline_text = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-overflowText.ipc-overflowText--pageSection.ipc-overflowText--base > div > div > div')
        storyline = storyline_text[0].text

        movies_data.loc[movies_data['title'] == name, 'storyline'] = storyline

    except IndexError:
        un_storyline.append(name)
        
    ################################################################################
    #age_rating
    try:
        age_rating = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > ul.ipc-metadata-list.ipc-metadata-list--dividers-all.sc-ab73d3b3-1.jzyXiW.ipc-metadata-list--base > li > div > ul > li > span')
        movies_data.loc[movies_data['title'] == name, 'age_rating'] = age_rating[0].text

    except:
        un_age_rating.append(name)
        
     ################################################################################
    # tags
    try:
        tags = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-ab73d3b3-0.iMuSjK > div.ipc-chip-list--base.ipc-chip-list.sc-e5ec7f64-0.iPSjrC > div.ipc-chip-list__scroller > a > span')
        movies_data.loc[movies_data['title'] == name, 'tag_1'] = tags[0].text
        movies_data.loc[movies_data['title'] == name, 'tag_2'] = tags[1].text
        movies_data.loc[movies_data['title'] == name, 'tag_3'] = tags[2].text
        
    except:
        un_tags.append(name)

    ################################################################################
            
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(1.5)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    

    #country
    try:
        country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
        movies_data.loc[movies_data['title'] == name, 'country'] = country.text

    except:
        un_country.append(name)
        
   ################################################################################
    # language
    try:
        language_select = ''
        language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')
    
        # 모든 <ul> 태그를 순회
        for ul in language_select:
            # 각 <ul> 태그 내의 모든 <li> 태그를 순회
            for li in ul.find_all('li'):
                span = li.find('span', class_='ipc-metadata-list-item__label')
                if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
                    # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
                    languages = [a.get_text(strip=True) for a in li.find_all('a')]
                    languages_str = ', '.join(languages)
                    break
            else:
                # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
                continue
            # 'Language' 라벨을 찾은 경우, 반복문 종료
            break
        
        movies_data.loc[movies_data['title'] == name, 'language'] = languages_str

    except:
        un_language.append(name)
        
     ################################################################################
    # director
    try:
        director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'director'] = director

    except:
        un_director.append(name)
        
     ################################################################################
    #release_year
    try:
        release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'release_year'] = release_year

    except:
        un_release_year.append(name)
        
     ################################################################################
    #runtime
    try:

        temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
        for tag in temp:
            if 'hour' in tag.text or 'minutes' in tag.text:
                runtime = tag.text
        
        movies_data.loc[movies_data['title'] == name, 'runtime'] = runtime

    except:
        un_runtime.append(name)
        

    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies:   1%|▌                                                             | 2/225 [00:27<50:21, 13.55s/movie]


KeyboardInterrupt: 

In [94]:
movies_data.loc[movies_data['language'] == 'None']

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
1660,2227,"Lodger: A Story of the London Fog, The (1927)",Crime|Drama|Thriller,"A serial killer known as ""The Avenger"" is on t...",United Kingdom,None,Alfred Hitchcock,1927,1 hour 32 minutes,15,serial killer,lodger,detective
2279,3022,"General, The (1926)",Comedy|War,"Johnnie loves his train (""The General"") and An...",United States,None,Clyde Bruckman,1926,1 hour 18 minutes,All,train,confederate army,civil war
2739,3677,Baraka (1992),Documentary,"Without words, cameras show us the world, with...",United States,None,Ron Fricke,1992,1 hour 36 minutes,12,human nature,environmental education,world culture
3294,4454,More (1998),Animation|Drama|Sci-Fi|IMAX,"A factory worker in a dark, gray world assembl...",United States,None,Mark Osborne,1998,6 minutes,Not Rated,claymation,stop motion animation,one word title
4386,6433,"Man with the Movie Camera, The (Chelovek s kin...",Documentary,This playful film is at once a documentary of ...,Soviet Union,None,Dziga Vertov,1929,1 hour 8 minutes,Not Rated,experimental film,avant garde,soviet union
6097,42191,Luxo Jr. (1986),Animation|Children,"A larger lamp watches while a smaller, younger...",United States,None,John Lasseter,1986,2 minutes,All,no dialogue,lamp,ball
6431,51573,Meshes of the Afternoon (1943),Fantasy,"On her way home, a solitary woman picks up a f...",United States,None,Maya Deren,1943,14 minutes,Not Rated,national film registry,flower,knife
6562,55061,Electroma (2006),Drama|Sci-Fi,Daft Punk's Electroma is an odyssey of two rob...,France,None,Thomas Bangalter,2006,1 hour 14 minutes,TV-PG,robot sci fi,existentialism,cyberpunk
7172,72104,Balance (1989),Animation|Drama|Mystery|Sci-Fi|Thriller,The setting is on a floating platform where a ...,Germany,None,Christoph Lauenstein,1989,7 minutes,NaN,balance,surrealism,stop motion
7184,72356,Partly Cloudy (2009),Animation|Children|Comedy|Fantasy,"Everyone knows that the stork delivers babies,...",United States,None,Peter Sohn,2009,6 minutes,G,cg animation,delivery stork,cgi animation


In [88]:
movies_data.loc[movies_data['country'] == 'France']

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
11,12,Dracula: Dead and Loving It (1995),Comedy|Horror,Another spoof from the mind of Mel Brooks. Thi...,France,"English, German",Mel Brooks,1995.0,1 hour 28 minutes,15,vampire,insane asylum,jonathan harker character
22,23,Assassins (1995),Action|Crime|Thriller,Robert Rath is a seasoned hitman who just want...,France,"English, Dutch, Spanish",Richard Donner,1995.0,2 hours 13 minutes,18,assassin,rival,gun fu
28,29,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,"Set in a dystopian society, someone is kidnapp...",France,"French, Cantonese",Marc Caro,1995.0,1 hour 52 minutes,15,steampunk fantasy,steampunk sci fi,orphanage
29,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama,Uncle Liu brings his cousin to Shanghai to wor...,France,Mandarin,Yimou Zhang,1995.0,1 hour 48 minutes,18,boy,1930s,NaN
50,55,Georgia (1995),Drama,Sadie is desperately looking up to her older s...,France,"English, Hebrew",Ulu Grosbard,1995.0,1 hour 55 minutes,18,sister sister estrangement,sister sister conflict,sister sister relationship
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9563,173619,Fugitives (1986),Comedy|Crime,"Coming out from jail, Lucas has decided to cha...",France,French,Francis Veber,1986.0,1 hour 35 minutes,12,france,police,police detective
9597,175705,Themroc (1973),Comedy|Horror,"Made without proper language, just gibberish a...",France,None,Claude Faraldo,1973,1 hour 50 minutes,NaN,brother sister incest,one breast exposed,female topless nudity
9631,179073,Male Hunt (1964),Comedy,Antoine is about to get married to Gisèle. His...,France,French,Édouard Molinaro,1964.0,1 hour 40 minutes,NaN,friend,france,married
9632,179119,The Death of Stalin (2017),Comedy,"In early-1953 Moscow, under the Great Terror's...",France,English,Armando Iannucci,2017.0,1 hour 47 minutes,15,soviet union,satire,russia


In [92]:
# 'country'가 'France'인 행을 필터링하고, 이들의 'language' 열에 대한 값의 빈도수 계산
language_counts = movies_data[movies_data['country'] == 'Germany']['language'].value_counts()

print(language_counts)

language
English                            80
German                             29
German, English                    13
English, Spanish                    9
English, French                     7
                                   ..
German, Russian, Polish, Hebrew     1
English, Latin                      1
German, English, Hungarian          1
German, English, Russian            1
German, French, Latin               1
Name: count, Length: 63, dtype: int64


3. release_year

In [95]:
movies_data[movies_data['release_year'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
569,695,True Crime (1996),Mystery|Thriller,"Mary Giordano is a bright, intelligent student...",United States,English,Pat Verducci,NaN,1 hour 34 minutes,"Rated R for psychopathic violence, strong sexu...",mentor,catholic school,case
590,728,Cold Comfort Farm (1995),Comedy,"In England in the early 1930s, twenty-year-old...",United Kingdom,English,John Schlesinger,NaN,1 hour 45 minutes,Rated PG for brief sensuality and language,female protagonist,female led relationship,f rated
673,888,Land Before Time III: The Time of the Great Gi...,Adventure|Animation|Children|Musical,When a meteorite had trapped all their water s...,United States,English,Roy Allen Smith,NaN,1 hour 11 minutes,All,punishment,father son relationship,villain
812,1064,Aladdin and the King of Thieves (1996),Animation|Children|Comedy|Fantasy|Musical|Romance,"At long last, Aladdin is about to marry the Pr...",United States,English,Tad Stones,NaN,1 hour 21 minutes,All,no opening credits,scene after end credits,surprise after end credits
842,1105,Children of the Corn IV: The Gathering (1996),Horror,All the kids in a town over night become fever...,United States,English,Greg Spence,NaN,1 hour 25 minutes,"Rated R for strong horror violence and gore, a...",scythe,reference to charles manson,corn
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9695,184791,Fred Armisen: Standup for Drummers (2018),Comedy,Comedian and drummer Fred Armisen performs sta...,English,"English, Spanish",Lance Bangs,NaN,1 hour 5 minutes,12,tv special,stand up comedy,stand up comedy special
9702,185135,Sherlock - A Study in Pink (2010),Crime,"Wounded in Afghanistan while in the Army, Dr. ...",United Kingdom,English,Paul McGuigan,NaN,1 hour 28 minutes,TV-14,man uses a cane,name calling,four word title
9726,190209,Jeff Ross Roasts the Border (2017),Comedy,Roastmaster Jeff Ross explores the world surro...,United States,English,Stuart Miller,NaN,1 hour 3 minutes,NaN,political satire,satire,political humor
9730,190221,Hommage à Zgougou (et salut à Sabine Mamou) (2...,Documentary,"A short tribute to Zgougou, Agnès Varda's cat.",France,French,Agnès Varda,NaN,2 minutes,NaN,cat,filmmaking,computer


In [98]:
null_rows = movies_data[movies_data['release_year'].isnull()]
names = null_rows['title'].tolist()


for i in names:
    if movies_data.loc[movies_data['title'] == i, 'release_year'].isnull().any():
        #movie_dataset.loc[movie_dataset['title'] == i, 'release_year'] = movie_dataset['title'].str.extract(r'\((\d{4})\)')

        extracted_year = movies_data.loc[movies_data['title'] == i, 'title'].str.extract(r'\((\d{4})\)').iloc[0,0]
        # 추출한 연도를 release_year 열에 저장
        movies_data.loc[movies_data['title'] == i, 'release_year'] = extracted_year

In [100]:
null_rows = movies_data[movies_data['release_year'].isnull()]
names = null_rows['title'].tolist()
movies_data[movies_data['release_year'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,Crime|Drama|Mystery,The detective understands that the encryption ...,Soviet Union,Russian,Igor Maslennikov,NaN,1 hour 4 minutes,NaN,based on short story,NaN,NaN
9448,167570,The OA,Drama|Fantasy|Mystery|Sci-Fi,Created and written by Brit Marling and Zal Ba...,United States,English,Zal Batmanglij,NaN,1 hour,18,near death experience,female frontal nudity,female nudity
9514,171495,Cosmos,Documentary,Astronomer Dr. Carl Sagan is host and narrator...,United States,English,Ann Druyan,NaN,13 hours,TV-PG,scientific philosophy,lightyear,educational
9515,171631,Maria Bamford: Old Baby,Documentary|Comedy,"Maria Bamford performs a rapid-fire, playful s...",United States,English,Jessica Yu,NaN,1 hour 4 minutes,18,stand up special,stand up comedy special,stand up comedy
9518,171749,Death Note: Desu nôto (2006–2007),Animation|Crime|Drama|Fantasy|Mystery|Thriller,After an intelligent yet cynical high school s...,Japan,Japanese,Mamoru Miyano,NaN,14 hours 11 minutes,15,battle of wits,anime,anti hero


In [101]:
names

['The Adventures of Sherlock Holmes and Doctor Watson',
 'The OA',
 'Cosmos',
 'Maria Bamford: Old Baby',
 'Death Note: Desu nôto (2006–2007)']

In [110]:
movies_data.loc[movies_data['title'] == names[4], 'release_year'] = '2006'

In [108]:
imdb_id = movies.loc[movies['title'] == names[4], 'imdbId'].values[0]
print(create_imdb_url(imdb_id))

https://www.imdb.com/title/tt6264596/


4. age_rating

In [111]:
movies_data[movies_data['age_rating'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
526,615,Bread and Chocolate (Pane e cioccolata) (1973),Comedy|Drama,Italian immigrant tries to become a member of ...,Italy,"Italian, German, English, French, Greek, Spanish",Franco Brusati,1974.0,1 hour 40 minutes,NaN,immigration,expatriate,satire comedy
544,645,Nelly & Monsieur Arnaud (1995),Drama,"When Nelly, a woman being just divorced, meets...",France,French,Claude Sautet,1995.0,1 hour 46 minutes,NaN,character names as title,older man young woman friendship,rich man poor woman
604,759,Maya Lin: A Strong Clear Vision (1994),Documentary,Documentary of the designing and creation of t...,United States,English,Freida Lee Mock,1994.0,1 hour 45 minutes,NaN,vietnam war veteran,biographical documentary,memorial
623,790,"Unforgettable Summer, An (Un été inoubliable) ...",Drama,Romania in the 1920s. Marie-Thérèse Von Debret...,Romania,"Romanian, French, English",Lucian Pintilie,1994.0,1 hour 22 minutes,NaN,border town,card game,goose
641,824,Kaspar Hauser (1993),Drama|Mystery,Kaspar Hauser is the son of the duke of Baden....,Germany,German,Peter Sehr,1993.0,2 hours 19 minutes,NaN,prison,duke,mute
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9726,190209,Jeff Ross Roasts the Border (2017),Comedy,Roastmaster Jeff Ross explores the world surro...,United States,English,Stuart Miller,2017,1 hour 3 minutes,NaN,political satire,satire,political humor
9728,190215,Liquid Truth (2017),Drama,A boy's parents accuse a swimming teacher of a...,Brazil,Portuguese,Carolina Jabor,2017.0,1 hour 27 minutes,NaN,swimming,teacher,father son relationship
9730,190221,Hommage à Zgougou (et salut à Sabine Mamou) (2...,Documentary,"A short tribute to Zgougou, Agnès Varda's cat.",France,French,Agnès Varda,2002,2 minutes,NaN,cat,filmmaking,computer
9734,193571,Silver Spoon (2014),Comedy|Drama,"Yuugo, who grew up in the city, finds himself ...",Japan,Japanese,Keisuke Yoshida,2014.0,1 hour 51 minutes,NaN,live action remake of anime,japan,farm life


In [112]:
null_rows = movies_data[movies_data['age_rating'].isnull()]
names = null_rows['title'].tolist()

In [114]:
names = null_rows['title'].tolist()

from tqdm import tqdm

#진행상황 확인을 위한
total_movies = len(names)

un_storyline = []
un_country = []
un_language = []
un_director = []
un_release_year = []
un_runtime = []
un_age_rating = []
un_tags=[]

for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    imdb_id = movies.loc[movies['title'] == name, 'imdbId'].values[0]
    imdb_url = create_imdb_url(imdb_id)
    # IMDb 페이지로 이동
    browser.get(imdb_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(1)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # 스크롤 내리기
    body = browser.find_elements('css selector', 'body')[0]
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    time.sleep(1.5)
    
    

    #country
    try:
        country = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul > li > div > ul > li > a')[1]
        movies_data.loc[movies_data['title'] == name, 'country'] = country.text

    except:
        un_country.append(name)
        
   ################################################################################
    # language
    try:
        language_select = ''
        language_select = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section > div.sc-f65f65be-0.bBlII > ul ')
    
        # 모든 <ul> 태그를 순회
        for ul in language_select:
            # 각 <ul> 태그 내의 모든 <li> 태그를 순회
            for li in ul.find_all('li'):
                span = li.find('span', class_='ipc-metadata-list-item__label')
                if span and ((span.get_text(strip=True) == 'Language') or (span.get_text(strip=True) == 'Languages' )):
                    # 'Language' 라벨의 부모 <li> 태그 내의 모든 <a> 태그에서 언어 추출
                    languages = [a.get_text(strip=True) for a in li.find_all('a')]
                    languages_str = ', '.join(languages)
                    break
            else:
                # 현재 <ul> 내에서 'Language' 라벨을 찾지 못한 경우, 다음 <ul>로 이동
                continue
            # 'Language' 라벨을 찾은 경우, 반복문 종료
            break
        
        movies_data.loc[movies_data['title'] == name, 'language'] = languages_str

    except:
        un_language.append(name)
        
     ################################################################################
    # director
    try:
        director = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div > section > section > div > div > div.sc-e226b0e3-10 > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'director'] = director

    except:
        un_director.append(name)
        
     ################################################################################
    #release_year
    try:
        release_year = soup.select('#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(1) > a')[0].text
        movies_data.loc[movies_data['title'] == name, 'release_year'] = release_year

    except:
        un_release_year.append(name)
        
     ################################################################################
    #runtime
    try:

        temp = soup.select("#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-a83bf66d-1.gYStnb.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.celwidget > div.sc-f65f65be-0.bBlII > ul.ipc-metadata-list.ipc-metadata-list--dividers-none.ipc-metadata-list--compact.ipc-metadata-list--base > li.ipc-metadata-list__item > div.ipc-metadata-list-item__content-container")
        for tag in temp:
            if 'hour' in tag.text or 'minutes' in tag.text:
                runtime = tag.text
        
        movies_data.loc[movies_data['title'] == name, 'runtime'] = runtime

    except:
        un_runtime.append(name)
        

    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies: 100%|████████████████████████████████████████████████████████████| 225/225 [25:24<00:00,  6.77s/movie]


In [121]:
movies_data[movies_data['tag_1'].isnull()]

,movieId,title,genres,storyline,country,language,director,release_year,runtime,age_rating,tag_1,tag_2,tag_3
208,242,Farinelli: il castrato (1994),Drama|Musical,Farinelli is the artistic name of Carlo Brosch...,France,"French, Italian",Gérard Corbiau,1994.0,1 hour 51 minutes,18,NaN,NaN,NaN
209,243,Gordy (1995),Children|Comedy|Fantasy,This is the story of a talking cute piglet nam...,United States,English,Mark Lewis,1994.0,1 hour 30 minutes,All,NaN,NaN,NaN
210,246,Hoop Dreams (1994),Documentary,This documentary follows two young African-Ame...,United States,English,Steve James,1994.0,2 hours 50 minutes,12,NaN,NaN,NaN
211,247,Heavenly Creatures (1994),Crime|Drama,Pauline Parker and Juliet Hulme met in school ...,New Zealand,"English, French",Peter Jackson,1994.0,1 hour 39 minutes,(Banned),NaN,NaN,NaN
212,248,Houseguest (1994),Comedy,Kevin Franklin is a con man whose mouth is big...,United States,English,Randall Miller,1995.0,1 hour 53 minutes,All,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8854,133115,We Could Be King (2014),Documentary,Germantown and Martin Luther King High Schools...,United States,English,Judd Ehrlich,2014.0,1 hour 20 minutes,Not Rated,NaN,NaN,NaN
9426,166024,Whiplash (2013),Short|Drama|Music,"A ferocious, bullying music teacher teaches a ...",English,English,Damien Chazelle,2013.0,18 minutes,TV-14,NaN,NaN,NaN
9544,172793,Vovka in the Kingdom of Far Far Away (1965),Adventure|Animation|Children|Fantasy,Schoolboy Vovka dreamed of a fabulous life. Wi...,Soviet Union,Russian,Boris Stepantsev,1965,19 minutes,Not Rated,NaN,NaN,NaN
9589,175435,The Magic Ring (1982),Animation|Fantasy,Ivan is a simple village boy who has absolutel...,Soviet Union,Russian,Leonid Nosyrev,1979,20 minutes,Not Rated,NaN,NaN,NaN


In [117]:
null_rows = movies_data[movies_data['age_rating'].isnull()]
names = null_rows['title'].tolist()

for i in names:
    if movies_data.loc[movies_data['title'] == i, 'age_rating'].isnull().any():
        movies_data.loc[movies_data['title'] == i, 'age_rating'] = 'Not Rated'

In [118]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       9742 non-null   int64 
 1   title         9742 non-null   object
 2   genres        9742 non-null   object
 3   storyline     9742 non-null   object
 4   country       9742 non-null   object
 5   language      9742 non-null   object
 6   director      9742 non-null   object
 7   release_year  9742 non-null   object
 8   runtime       9742 non-null   object
 9   age_rating    9742 non-null   object
 10  tag_1         9446 non-null   object
 11  tag_2         9411 non-null   object
 12  tag_3         9393 non-null   object
dtypes: int64(1), object(12)
memory usage: 989.5+ KB


In [120]:
movies_data.to_csv("../../dataset/movies_crawling_result.csv", index=False)

In [122]:
null_rows = movies_data[movies_data['tag_1'].isnull()]
names = null_rows['title'].tolist()

In [137]:
def create_tmdb_url(tmdbId):
    # imdbId를 문자열로 변환하고, 7자리가 되도록 앞에 0을 추가
    # URL 생성
    url = f"https://www.themoviedb.org/movie/{tmdbId}"
    return url

In [128]:
tmdbId = int(movies.loc[movies['title'] == names[0], 'tmdbId'].values[0])

In [130]:
int(tmdbId)

10954

In [132]:
null_df = pd.read_csv("../../dataset/1210_dfNa.csv")
null_df.head()

,movieId,title
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
1,242,Farinelli: il castrato (1994)
2,243,Gordy (1995)
3,250,Heavyweights (Heavy Weights) (1995)
4,251,"Hunted, The (1995)"


In [133]:
names = null_df['title'].tolist()

In [140]:
tmdbId = int(movies.loc[movies['title'] == names[0], 'tmdbId'].values[0])
tmdb_link = create_tmdb_url(tmdbId)

# 1개로 크롤링

In [139]:
browser = webdriver.Chrome()
browser.get('https://www.google.com/')

soup = BeautifulSoup(browser.page_source, 'html.parser')

In [141]:
browser.get(tmdb_link)

In [142]:
soup = BeautifulSoup(browser.page_source, 'html.parser')
a = soup.select('#media_v4 > div > div > div.grey_column > div > section > div > div > section.keywords.right_column > ul > li > a')

In [143]:
len(a)

12

In [146]:
a[0].text

'shanghai, china'

In [147]:
a[1].text

'chinese mafia'

In [148]:
a[2].text

'coming of age'

In [159]:
null_df

,movieId,title
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
1,242,Farinelli: il castrato (1994)
2,243,Gordy (1995)
3,250,Heavyweights (Heavy Weights) (1995)
4,251,"Hunted, The (1995)"
...,...,...
261,176601,Black Mirror
262,180777,Die Frauen von Ravensbrück (2005)
263,193567,anohana: The Flower We Saw That Day - The Movi...
264,193579,Jon Stewart Has Left the Building (2015)


## 크롤링 let's go!

In [161]:
from tqdm import tqdm

for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    tmdbId = int(movies.loc[movies['title'] == name, 'tmdbId'].values[0])
    tmdb_link = create_tmdb_url(tmdbId)
  

    browser.get(tmdb_link)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

     ################################################################################
    # tag1
    try:
        tags = soup.select('#media_v4 > div > div > div.grey_column > div > section > div > div > section.keywords.right_column > ul > li > a')

        null_df.loc[null_df['title'] == name, 'tag_1'] = tags[0].text
        null_df.loc[null_df['title'] == name, 'tag_2'] = tags[1].text
        null_df.loc[null_df['title'] == name, 'tag_3'] = tags[2].text
        
        tags = ''
    except:
        soup = BeautifulSoup(browser.page_source, 'html.parser')



    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies:  77%|██████████████████████████████████████████████▍             | 206/266 [05:57<01:43,  1.73s/movie]


ValueError: cannot convert float NaN to integer

In [162]:
null_df

,movieId,title,tag_1,tag_2,tag_3
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,"shanghai, china",chinese mafia,coming of age
1,242,Farinelli: il castrato (1994),sibling relationship,opera,eunuch
2,243,Gordy (1995),country music,pig,talking pig
3,250,Heavyweights (Heavy Weights) (1995),overweight,summer camp,psychopath
4,251,"Hunted, The (1995)",martial arts,assassin,self sacrifice
...,...,...,...,...,...
261,176601,Black Mirror,NaN,NaN,NaN
262,180777,Die Frauen von Ravensbrück (2005),NaN,NaN,NaN
263,193567,anohana: The Flower We Saw That Day - The Movi...,NaN,NaN,NaN
264,193579,Jon Stewart Has Left the Building (2015),NaN,NaN,NaN


In [163]:
null_rows = null_df[null_df['tag_1'].isnull()]
names = null_rows['title'].tolist()

In [164]:
from tqdm import tqdm

for name in tqdm(names, desc="Crawling movies", unit="movie"):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    try:
        tmdbId = int(movies.loc[movies['title'] == name, 'tmdbId'].values[0])
        tmdb_link = create_tmdb_url(tmdbId)
    
    except ValueError:
        continue

    browser.get(tmdb_link)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

     ################################################################################
    # tag1
    try:
        tags = soup.select('#media_v4 > div > div > div.grey_column > div > section > div > div > section.keywords.right_column > ul > li > a')

        null_df.loc[null_df['title'] == name, 'tag_1'] = tags[0].text
        null_df.loc[null_df['title'] == name, 'tag_2'] = tags[1].text
        null_df.loc[null_df['title'] == name, 'tag_3'] = tags[2].text
        
        tags = ''
    except:
        soup = BeautifulSoup(browser.page_source, 'html.parser')



    ################################################################################
            
    soup = BeautifulSoup(browser.page_source, 'html.parser')

Crawling movies: 100%|██████████████████████████████████████████████████████████████| 78/78 [02:10<00:00,  1.68s/movie]


In [167]:
temp_df = null_df[null_df['tag_1'].isnull()]
temp_df.shape

(36, 5)

In [168]:
null_df.to_csv("1210_dfNa_tag.csv", index = False)

In [170]:
temp = pd.read_csv("../../dataset/ratings.csv")
temp.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [176]:
unique_user_ids = temp['userId'].unique()
len(unique_user_ids)

610

In [179]:
df = pd.read_csv("../../dataset/links.csv")
df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
